# Introduction

Notebook para avaliar desempenho de BiopassID com relação aos requisitos ICAO.

Nós usamos o dataset FVC NOT ALIGNED, concatenando os splits de train, valid e test.

In [1]:
%load_ext autoreload
%autoreload 2

# Import Libraries

In [2]:
import os
import sys
import cv2
import base64
import shutil
import pprint
import requests
import pandas as pd

if '../../../../notebooks/' not in sys.path:
    sys.path.insert(0, '../../../../notebooks/')

from net_data_loaders.net_gt_loader import NetGTLoader
from gt_loaders.gt_names import GTName
from utils.constants import ICAO_REQ
from utils.vsoft_gt import COLS
from gt_loaders.gen_gt import Eval

# Loading Data

In [3]:
net_gt_loader = NetGTLoader(aligned=False, gt_names=[GTName.FVC], requisites=list(ICAO_REQ), is_mtl_model=False)
train_data = net_gt_loader.load_gt_data(split='train')
valid_data = net_gt_loader.load_gt_data(split='validation')
test_data = net_gt_loader.load_gt_data(split='test')

all_data = pd.concat([train_data, valid_data, test_data], ignore_index=True)

print(all_data.shape)

Loading GT FVC - TRAIN split...
..Ignoring 0 empty label values
Input data.shape: (4928, 26)
Loading GT FVC - VALIDATION split...
..Ignoring 0 empty label values
Input data.shape: (547, 26)
Loading GT FVC - TEST split...
..Ignoring 0 empty label values
Input data.shape: (288, 26)
(5763, 26)


In [4]:
all_data.head()

,origin,img_name,mouth,rotation,l_away,eyes_closed,close,hat,dark_glasses,frames_heavy,...,sh_head,blurred,ink_mark,skin_tone,washed_out,pixelation,hair_eyes,background,red_eyes,aligned
0,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,False
1,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,False
2,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,False
3,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,1,1,1,1,1,1,...,0,1,1,0,1,1,1,0,1,False
4,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,1,1,1,1,1,1,...,0,1,1,0,1,1,1,0,1,False


# Utilitary Functions

In [5]:
def make_request(img_path):
    encoded_string = None
    with open(img_path, "rb") as image_file:
        buffer = image_file.read()
        encoded_img = base64.b64encode(buffer).decode("utf-8")

    url = f'https://api.biopassid.com/quality/check_icao'
    headers = {'Content-Type': 'application/json',
               'Ocp-Apim-Subscription-Key': 'bba14916cb1d4b35a6e8e2f9d2bd3fe6'}

    json_dict = {'Check': { 'Image': encoded_img }}
    response = requests.post(url, headers=headers, data=str(json_dict))
    
    if response.status_code == 200:
        print(' .. Request OK')
        data = response.json()['ICAO']['Requirements']
        return data
    else:
        print(' .. Bad Request')
        return None

# Submit Images to BiopassID

In [6]:
raw_data_df = pd.DataFrame(columns=['origin']+list(COLS.values()))

for idx, img_path in enumerate(all_data.img_name.values[:50]):
    print(idx, img_path)
    
    data = make_request(img_path)
    
    new_row = {}
    if data is None:
        for req_name in COLS.values():
            new_row[req_name] = str(Eval.NO_ANSWER.value)
    else:
        for req,v in data.items():
            req_name = COLS[req]
            compliant = str(Eval.COMPLIANT.value) if v['IsConformed'] else str(Eval.NON_COMPLIANT.value)
            new_row[req_name] = compliant

    new_row['img_name'] = img_path
    new_row['origin'] = 'VSOFT'
    raw_data_df = raw_data_df.append(new_row, ignore_index=True) 
    raw_data_df.to_csv('vsoft_data/compliance_check_vsoft.csv', index=False)    
    
raw_data_df.head(10)

0 /home/guilherme/data1/Dropbox/Link to Desktop/Doutorado/Datasets/icao_dataset/train/FRGC_04288d208.jpg
 .. Request OK
1 /home/guilherme/data1/Dropbox/Link to Desktop/Doutorado/Datasets/icao_dataset/train/FRGC_04233d404.jpg
 .. Request OK
2 /home/guilherme/data1/Dropbox/Link to Desktop/Doutorado/Datasets/icao_dataset/train/FRGC_04549d72.jpg
 .. Request OK
3 /home/guilherme/data1/Dropbox/Link to Desktop/Doutorado/Datasets/icao_dataset/train/FRGC_04324d04.jpg
 .. Request OK
4 /home/guilherme/data1/Dropbox/Link to Desktop/Doutorado/Datasets/icao_dataset/train/FRGC_04323d78.jpg
 .. Request OK
5 /home/guilherme/data1/Dropbox/Link to Desktop/Doutorado/Datasets/icao_dataset/train/AR_FDB_m-019-11.png
 .. Request OK
6 /home/guilherme/data1/Dropbox/Link to Desktop/Doutorado/Datasets/icao_dataset/train/FRGC_04279d135.jpg
 .. Request OK
7 /home/guilherme/data1/Dropbox/Link to Desktop/Doutorado/Datasets/icao_dataset/train/visio_icao_expotec_843.png
 .. Request OK
8 /home/guilherme/data1/Dropbox/Li

,origin,img_name,mouth,rotation,l_away,eyes_closed,close,hat,dark_glasses,frames_heavy,...,sh_face,sh_head,blurred,ink_mark,skin_tone,washed_out,pixelation,hair_eyes,background,red_eyes
0,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,0,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
3,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,0,0,1
4,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,1,1,1,1,1,1,...,0,1,0,1,1,1,1,1,0,1
5,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,0,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
6,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,0,1
8,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,-99,-99,-99,-99,-99,-99,-99,-99,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
9,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,1,1,1


# Load Data

In [7]:
data_df = pd.read_csv('vsoft_data/compliance_check_vsoft.csv')
data_df.head()

,origin,img_name,mouth,rotation,l_away,eyes_closed,close,hat,dark_glasses,frames_heavy,...,sh_face,sh_head,blurred,ink_mark,skin_tone,washed_out,pixelation,hair_eyes,background,red_eyes
0,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,0,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
3,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,0,0,1
4,VSOFT,/home/guilherme/data1/Dropbox/Link to Desktop/...,1,1,1,1,1,1,1,1,...,0,1,0,1,1,1,1,1,0,1
